In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


DATA_PATH = '../application_train_FinalEDA.csv'
RISK_PROFILE_TRAIN_OUT = "risk_profiled_autoencoder_train.csv"
AE_TEST_FEATURES_OUT = "autoencoder_test_features.csv"
SUMMARY_OUT = "evaluation_results_summary.csv"
RANDOM_STATE = 42


df1 = pd.read_csv(DATA_PATH)
print("Dataset loaded successfully!")
print("Shape:", df1.shape)
print("\nTARGET distribution:")
print(df1['TARGET'].value_counts(normalize=True))


X = df1.drop(columns=['TARGET'])
y = df1['TARGET']

# TRAIN-TEST SPLIT (STRATIFIED)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("\nTrain TARGET distribution:\n", y_train.value_counts(normalize=True))
print("\nTest TARGET distribution:\n", y_test.value_counts(normalize=True))

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures scaled successfully!")

# DEFINING AND TRAINING AUTOENCODER
input_dim = X_train_scaled.shape[1]
encoding_dim = 32

# Encoder-Decoder architecture
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
bottleneck = Dense(encoding_dim, activation='relu', name="bottleneck")(encoded)
decoded = Dense(64, activation='relu')(bottleneck)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
encoder = Model(inputs=input_layer, outputs=bottleneck)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

autoencoder.summary()

# Training autoencoder on NON-DEFAULTERS only (anomaly-detection style)
X_train_ae = X_train_scaled[y_train.values == 0]
if X_train_ae.shape[0] < 10:
    print("Warning: very few non-defaulters in training set; autoencoder training may be unstable.")


if (y_test.values == 0).sum() > 0:
    X_val_ae = X_test_scaled[y_test.values == 0]
else:
    X_val_ae = X_test_scaled  # fallback

history = autoencoder.fit(
    X_train_ae, X_train_ae,
    epochs=30,
    batch_size=256,
    validation_data=(X_val_ae, X_val_ae),
    verbose=1
)

# Plot loss curves
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Autoencoder Training Loss (MSE)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# EXTRACTING ENCODED (LATENT) FEATURES
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)

encoded_train_df = pd.DataFrame(X_train_encoded, columns=[
                                f"encoded_{i}" for i in range(encoding_dim)])
encoded_test_df = pd.DataFrame(X_test_encoded, columns=[
                               f"encoded_{i}" for i in range(encoding_dim)])

encoded_train_df['TARGET'] = y_train.reset_index(drop=True)
encoded_test_df['TARGET'] = y_test.reset_index(drop=True)

print("\nEncoded features created successfully!")
print("Train encoded shape:", encoded_train_df.shape)
print("Test encoded shape:", encoded_test_df.shape)

# K-MEANS CLUSTERING (RISK PROFILING)
X_encoded = encoded_train_df.drop(columns=['TARGET'])
y_encoded = encoded_train_df['TARGET']

kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE)
encoded_train_df['Cluster'] = kmeans.fit_predict(X_encoded)

cluster_summary = (
    encoded_train_df.groupby('Cluster')['TARGET']
    .mean()
    .reset_index()
    .sort_values(by='TARGET')
)
cluster_summary.columns = ['Cluster', 'Average_Default_Rate']

print("\n Default Rate by Cluster:")
print(cluster_summary)

# LABEL CLUSTERS AS RISK LEVELS
risk_map = cluster_summary.sort_values(
    'Average_Default_Rate').reset_index(drop=True)
risk_map['Risk_Level'] = ['Low Risk', 'Medium Risk', 'High Risk']
risk_label_dict = dict(zip(risk_map['Cluster'], risk_map['Risk_Level']))

encoded_train_df['Risk_Level'] = encoded_train_df['Cluster'].map(
    risk_label_dict)

print("\n Cluster → Risk Level mapping:")
print(risk_label_dict)
print("\n Risk Level Distribution:")
print(encoded_train_df['Risk_Level'].value_counts())

# VISUALIZE CLUSTERS USING PCA
pca = PCA(n_components=2, random_state=RANDOM_STATE)
pca_result = pca.fit_transform(X_encoded)

encoded_train_df['PCA1'] = pca_result[:, 0]
encoded_train_df['PCA2'] = pca_result[:, 1]

plt.figure(figsize=(8, 6))
for level in encoded_train_df['Risk_Level'].unique():
    subset = encoded_train_df[encoded_train_df['Risk_Level'] == level]
    plt.scatter(subset['PCA1'], subset['PCA2'], label=level, alpha=0.6)
plt.title("PCA Projection of Autoencoder Risk Clusters")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.legend()
plt.show()

# SAVE THE RISK-PROFILED DATASET
encoded_train_df.to_csv(RISK_PROFILE_TRAIN_OUT, index=False)
encoded_test_df.to_csv(AE_TEST_FEATURES_OUT, index=False)

print("\nSaved successfully:")
print("→", RISK_PROFILE_TRAIN_OUT)
print("→", AE_TEST_FEATURES_OUT)

# EVALUATION: Reconstruction metrics, distribution, ROC-AUC, thresholding, confusion matrix

# Reconstruct test set using autoencoder
X_test_recon = autoencoder.predict(X_test_scaled)
# Per-sample reconstruction error (MSE per sample)
test_errors = np.mean(np.square(X_test_scaled - X_test_recon), axis=1)

# Reconstruction on training non-defaulters used for AE training (for thresholding & distribution)
X_train_ae_recon = autoencoder.predict(X_train_ae)
train_errors = np.mean(np.square(X_train_ae - X_train_ae_recon), axis=1)

# Aggregated reconstruction metrics (on full test set)
recon_mse = mean_squared_error(X_test_scaled, X_test_recon)
recon_mae = mean_absolute_error(X_test_scaled, X_test_recon)

print("\n--- Autoencoder Reconstruction Metrics (on test set) ---")
print(f"Reconstruction MSE (test, aggregated): {recon_mse:.6f}")
print(f"Reconstruction MAE (test, aggregated): {recon_mae:.6f}")

# Plot reconstruction error distribution (train non-defaulters)
plt.figure(figsize=(8, 5))
plt.hist(train_errors, bins=80, alpha=0.6)
plt.title('Reconstruction error distribution (train non-defaulters)')
plt.xlabel('Reconstruction error (MSE per sample)')
plt.ylabel('Count')
plt.show()

# reconstruction error histogram (overlay)
plt.figure(figsize=(8, 5))
plt.hist(train_errors, bins=80, alpha=0.5, label='train non-defaulters')
plt.hist(test_errors, bins=80, alpha=0.5, label='test (all)')
plt.title('Reconstruction error: train (non-defaulters) vs test')
plt.xlabel('Reconstruction error (MSE per sample)')
plt.ylabel('Count')
plt.legend()
plt.show()

# ROC-AUC using reconstruction error as anomaly score
roc_auc = roc_auc_score(y_test, test_errors)
fpr, tpr, thresholds = roc_curve(y_test, test_errors)
print(f"\nROC-AUC (autoencoder reconstruction error): {roc_auc:.4f}")

# Chose threshold via Youden's J statistic
j_scores = tpr - fpr
j_idx = np.argmax(j_scores)
opt_threshold = thresholds[j_idx]
# 95th percentile of training non-defaulters
pct_threshold = np.percentile(train_errors, 95)

print(f"Optimal threshold (Youden): {opt_threshold:.6f}")
print(f"95th percentile threshold (train errors): {pct_threshold:.6f}")

# Binary predictions using chosen threshold (Youden)
y_pred_ae = (test_errors >= opt_threshold).astype(int)

prec_ae = precision_score(y_test, y_pred_ae, zero_division=0)
rec_ae = recall_score(y_test, y_pred_ae, zero_division=0)
f1_ae = f1_score(y_test, y_pred_ae, zero_division=0)
cm_ae = confusion_matrix(y_test, y_pred_ae)

print("\n--- Autoencoder Classification-style Metrics (using Youden threshold) ---")
print("Confusion matrix:\n", cm_ae)
print(f"Precision: {prec_ae:.4f}, Recall: {rec_ae:.4f}, F1: {f1_ae:.4f}")

plt.figure(figsize=(6, 5))
plt.imshow(cm_ae, interpolation='nearest')
plt.title("Confusion Matrix (Autoencoder - Youden threshold)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
for i in range(cm_ae.shape[0]):
    for j in range(cm_ae.shape[1]):
        plt.text(j, i, cm_ae[i, j], ha="center", va="center", fontsize=12)
plt.colorbar()
plt.show()


plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}")
plt.plot([0, 1], [0, 1], linestyle='--', linewidth=0.8)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Autoencoder reconstruction error)")
plt.legend()
plt.grid(True)
plt.show()


precision_vals, recall_vals, pr_thresholds = precision_recall_curve(
    y_test, test_errors)
ap_score = average_precision_score(y_test, test_errors)
plt.figure(figsize=(7, 5))
plt.plot(recall_vals, precision_vals, label=f"AP = {ap_score:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve (Autoencoder)")
plt.legend()
plt.grid(True)
plt.show()

# COMPARISON WITH SUPERVISED MODELS
print("\nTraining supervised classifiers for comparison...")

lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)

lr.fit(X_train_scaled, y_train)
rf.fit(X_train_scaled, y_train)

# Probabilities and preds on test
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
y_pred_lr = lr.predict(X_test_scaled)

y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]
y_pred_rf = rf.predict(X_test_scaled)

# Metrics
auc_lr = roc_auc_score(y_test, y_proba_lr)
prec_lr = precision_score(y_test, y_pred_lr, zero_division=0)
rec_lr = recall_score(y_test, y_pred_lr, zero_division=0)
f1_lr = f1_score(y_test, y_pred_lr, zero_division=0)

auc_rf = roc_auc_score(y_test, y_proba_rf)
prec_rf = precision_score(y_test, y_pred_rf, zero_division=0)
rec_rf = recall_score(y_test, y_pred_rf, zero_division=0)
f1_rf = f1_score(y_test, y_pred_rf, zero_division=0)

print('\n--- Supervised models metrics (on same test set) ---')
print('Logistic Regression: AUC={:.4f}, Precision={:.4f}, Recall={:.4f}, F1={:.4f}'.format(
    auc_lr, prec_lr, rec_lr, f1_lr))
print('Random Forest:        AUC={:.4f}, Precision={:.4f}, Recall={:.4f}, F1={:.4f}'.format(
    auc_rf, prec_rf, rec_rf, f1_rf))

# Save summary table
results = pd.DataFrame({
    'model': ['Autoencoder (AE error->AUC)', 'Autoencoder (thresholded)', 'LogisticRegression', 'RandomForest'],
    'roc_auc': [roc_auc, roc_auc, auc_lr, auc_rf],
    'precision': [np.nan, prec_ae, prec_lr, prec_rf],
    'recall': [np.nan, rec_ae, rec_lr, rec_rf],
    'f1': [np.nan, f1_ae, f1_lr, f1_rf]
})
results.to_csv(SUMMARY_OUT, index=False)
print(f"\nAll summary results saved to {SUMMARY_OUT}")


print("\n--- Final summary ---")
print(results)


test_results_df = pd.DataFrame({
    'reconstruction_error': test_errors,
    'y_true': y_test.reset_index(drop=True),
    'y_pred_ae_youden': y_pred_ae
})
test_results_df.to_csv("autoencoder_test_errors_with_labels.csv", index=False)
print("Saved test errors to autoencoder_test_errors_with_labels.csv")

Dataset loaded successfully!
Shape: (307511, 57)

TARGET distribution:
TARGET
0    0.919271
1    0.080729
Name: proportion, dtype: float64

Train TARGET distribution:
 TARGET
0    0.919271
1    0.080729
Name: proportion, dtype: float64

Test TARGET distribution:
 TARGET
0    0.919272
1    0.080728
Name: proportion, dtype: float64

Features scaled successfully!


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │         7,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bottleneck (Dense)              │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 56)             │         7,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,288 (137.84 KB)

 Trainable params: 35,288 (137.84 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
860/884 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1934